In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Generate a random matrix A
n = 1000
A = np.random.normal(size=(n,n))
A = A.T @ A

In [3]:
sample_size = 100000

diag_estimate = np.zeros(n)
tk = np.zeros(n)
qk = np.zeros(n)

for j in range(sample_size):
    
    # Draw random vector
    vk = np.random.choice([-1, 1], size=n)

    # Update tk
    tk = tk + ((A @ vk) * vk)

    # Update qk
    qk = qk + (vk*vk)

    # Update diag_estimate
    diag_estimate = tk / qk

In [4]:
(np.linalg.norm( np.diag(A) - diag_estimate , ord=2)**2)/n

9.496848540707365

In [ ]:
def naive_diag(A, sample_size=1000):
    """Naive unbiased estimator for the diagonal of a matrix, see [5].
    """

    # Get shape
    n = A.shape[0]

    diag_estimate = np.zeros(n)
    tk = np.zeros(n)
    qk = np.zeros(n)

    for j in range(sample_size):
        
        # Draw random vector
        vk = np.random.choice([-1, 1], size=n)

        # Update tk
        tk = tk + ((A @ vk) * vk)

        # Update qk
        qk = qk + (vk*vk)

        # Update diag_estimate
        diag_estimate = tk / qk

    return diag_estimate

# Explicit probe?

In [10]:
def explicit_diag_probe(A):
    """Computes the diagonal of A using an explicit probe. Requires exactly n matvecs and rmatvecs with A.
    """

    # Setup
    n = A.shape[0]
    diagonal = np.zeros(n)

    for j in range(n):

        # jth column of the identity
        w = np.zeros(n)
        w[j] = 1.0

        # Compute w^T A w
        diagonal[j] = w.T @ A @ w

    return diagonal

In [12]:
explicit_diag_probe(A) - np.diag(A)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# Hadamard matrices method?

In [6]:
from scipy.linalg import hadamard as scipy_hadamard

In [7]:
s = 5
H = scipy_hadamard(2**4)[:,:5]

In [8]:
H

array([[ 1,  1,  1,  1,  1],
       [ 1, -1,  1, -1,  1],
       [ 1,  1, -1, -1,  1],
       [ 1, -1, -1,  1,  1],
       [ 1,  1,  1,  1, -1],
       [ 1, -1,  1, -1, -1],
       [ 1,  1, -1, -1, -1],
       [ 1, -1, -1,  1, -1],
       [ 1,  1,  1,  1,  1],
       [ 1, -1,  1, -1,  1],
       [ 1,  1, -1, -1,  1],
       [ 1, -1, -1,  1,  1],
       [ 1,  1,  1,  1, -1],
       [ 1, -1,  1, -1, -1],
       [ 1,  1, -1, -1, -1],
       [ 1, -1, -1,  1, -1]])

In [9]:
s = 20
m = 5
n = 4

H1 = scipy_hadamard(m)
H2 = scipy_hadamard(n)

ValueError: n must be an positive integer, and n must be a power of 2

In [165]:
import math
def closest_power2(k):
    """
    Returns the closest power of 2 by checking whether 
    the second binary number is a 1.

    k must be an integer.

    From https://stackoverflow.com/a/56225940/18096318.
    """
    op = math.floor if bin(k)[3] != "1" else math.ceil
    return 2**(op(math.log(k,2)))

def nearest_power2(k):
    """
    Returns the smallest power of 2 greater than or equal to k.

    From https://www.geeksforgeeks.org/smallest-power-of-2-greater-than-or-equal-to-n/.
    """
    a = int(math.log2(k))
    if 2**a == k:
        return k
    return 2**(a + 1)

def _hadamard_matrices_compute_submatrix_dimensions(n):
    """Given an integer s, computes integers m and n that are both powers of 2
    such that m*n is >= than s, but not by much.
    """ 

    p = nearest_power2(int(np.ceil(np.sqrt(n))))

    return p, p


def _hadamard_matrices_compute_submatrix_dimensions2(n, s):
    """
    """ 
    p = nearest_power2(n)
    q = nearest_power2(int(np.ceil(n/p)))
    return p, q

In [166]:
n = 100

In [167]:
n1, n2 = _hadamard_matrices_compute_submatrix_dimensions(n)

In [168]:
H1 = scipy_hadamard(n1)
H2 = scipy_hadamard(n2)
H3 = np.kron(H1, H2)

In [176]:
def get_kth_hadamard_row(H1, H2, k):
    """Given H1 and H2, computes the kth row of kron(H1, H2).
    """
    
    # Size of H2 matrix
    h1_size = H1.shape[0]
    h2_size = H2.shape[0]

    assert 0 <= k <= (h1_size*h2_size) - 1, "k outside of valid range."

    # Compute row index of H1 and H2
    h2_row_idx = k % h2_size
    h1_row_idx = int( np.floor( k / h1_size ) )
    
    return np.kron(H1[h1_row_idx,:], H2[h2_row_idx, :])

In [261]:
def _hadamard_build_V(n, s):
    """Given n and s, builds the matrix V."""
    n1, n2 = _hadamard_matrices_compute_submatrix_dimensions(n)
    H1 = scipy_hadamard(n1)
    H2 = scipy_hadamard(n2)
    V = np.zeros((n,s))
    for j in range(s):
        V[:,j] = get_kth_hadamard_row(H1, H2, j)[:n]

    return V

In [260]:
n = 128
s = 48
hadamard_build_V(n,s) @ hadamard_build_V(n,s).T

array([[48.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 48.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., 48., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., 48.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0., 48.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., 48.]])

In [263]:
def hadamard_diag(A, sample_size=1000):
    """Estimates the diagonal of A using a Hadard matrix method, see [5]. A must be SPD.
    """


    #assert (sample_size & (sample_size-1) == 0) and sample_size != 0, "sample_size must be a power of 2!"

    # Get shape
    n = A.shape[0]

    assert sample_size <= n, "sample size must be <= n."

    # Setup
    diag_estimate = np.zeros(n)
    tk = np.zeros(n)
    qk = np.zeros(n)

    n1, n2 = _hadamard_matrices_compute_submatrix_dimensions(n)
    H1 = scipy_hadamard(n1)
    H2 = scipy_hadamard(n2)

    for j in range(sample_size):
        
        # Get hadamard vector
        vk = get_kth_hadamard_row(H1, H2, j)[:n]

        # Update tk
        tk = tk + ((A @ vk) * vk)

        # Update qk
        qk = qk + (vk*vk)

        # Update diag_estimate
        diag_estimate = tk / qk

    return diag_estimate

In [266]:
# Generate a random matrix A
n = 128
A = np.random.normal(size=(n,n))
A = A.T @ A

In [267]:
hadamard_diag(A, 16)

array([163.09757774, 157.61306422, 137.96433791, 135.82807379,
       119.16116111, 100.44758867,  86.20726074,  45.48321725,
       138.8904248 , 140.93727147,  98.82908837, 153.68644362,
       117.28190781, 150.71069669, 105.16810459, 140.01111481,
       218.77485428, 111.68854693, 137.9411435 ,  76.40562161,
        96.12904335, 144.45610541,  91.45150646, 151.07592014,
       139.04570043, 169.70656426, 149.15804811, 179.79696735,
       176.30766569, 121.44062936, 142.12689889, 129.27520243,
       156.20155562, 119.58590216, 144.43895193, 107.51840271,
        90.52773525,  91.83076263,  79.52996094, 132.78024744,
       175.31995535,  90.83949117, 129.2351412 , 152.58523508,
       110.82489754,  84.7884165 , 158.85398509, 138.43895778,
        72.74902595, 130.22805815,  81.08187048, 137.62281494,
       137.92270667, 145.1534245 ,  67.68802499,  74.66562994,
       182.76652141,  80.19863807, 111.67255151, 123.4610997 ,
       120.27366673, 114.35068846, 155.13859697, 114.62

In [265]:
np.diag(A)

array([1009.79821029,  982.06643977,  954.14108269, 1049.67962967,
        914.14765863,  992.29715848,  963.86533743,  993.02962389,
        945.94982195,  960.85334756, 1041.06200346,  965.49887415,
        972.68909847,  956.29860097, 1012.62803431, 1050.20555523,
        976.31112909,  904.09958909, 1009.90799861,  977.62821238,
       1021.09379383,  988.26287491, 1004.6023719 , 1049.5565516 ,
       1007.67943632, 1011.80747903, 1050.00961991,  966.51887217,
       1013.86724107, 1021.21925402, 1000.60396178, 1095.29261934,
       1010.87353641, 1003.65155086, 1099.28387283, 1061.52527513,
       1005.61984716,  971.07475876,  981.84053714, 1001.83392145,
       1044.59634536,  987.41066612,  947.3938878 , 1009.80854958,
        945.9690068 , 1002.15402034,  899.24207595,  997.2177141 ,
       1001.15627782,  928.41861402,  978.51348781, 1054.11405482,
       1107.04445542,  963.47968801,  966.24915187,  925.92581182,
        974.28582519,  951.14058996, 1040.02273794, 1072.22132

In [264]:
hadamard_diag(A, sample_size=5)

array([ 6.30234256e+02,  1.09505313e+03,  9.60157869e+02,  1.27644576e+03,
        4.78308021e+02,  2.12815936e+03,  3.22041526e+02,  7.49772623e+02,
        1.42158652e+03,  1.59649096e+03,  1.12844825e+03,  1.49675351e+03,
        1.63450539e+03,  1.09428522e+03,  1.25085665e+03,  1.06938015e+03,
        1.33274724e+03,  1.58316942e+03,  1.26292474e+03,  1.26003068e+03,
        7.69470914e+02,  1.19626734e+03,  1.40590531e+03,  2.05224163e+02,
        1.43163313e+03,  8.95813065e+02,  6.25186697e+01,  9.00373769e+02,
        1.47541653e+03,  1.07986621e+03,  4.47004260e+02,  8.82234038e+02,
        1.16061835e+03,  6.33938830e+02,  3.64683241e+02,  7.08757716e+02,
        9.24748637e+02,  4.65293350e+02,  9.28500174e+02,  8.07379023e+02,
        9.42106305e+02,  8.14228019e+02,  9.17366543e+02,  8.85550090e+02,
        8.75517072e+02,  5.01731500e+02,  6.03803526e+02,  6.58429295e+02,
        1.52936144e+02,  9.21877611e+02,  8.87049046e+02,  1.13882005e+03,
        1.17186843e+03,  

In [23]:
H.T @ H

array([[16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 16,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 16,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 16,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 16,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 16,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 16,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

In [ ]:
def hadamard_diag(A, sample_size=1000):
    """Estimates the diagonal of A using a Hadard matrix method, see [5]. A must be SPD.

    sample_size must be a power of 2!
    """

    assert (sample_size & (sample_size-1) == 0) and sample_size != 0, "sample_size must be a power of 2!"

    # Get shape
    n = A.shape[0]

    # Setup
    diag_estimate = np.zeros(n)
    tk = np.zeros(n)
    qk = np.zeros(n)

    for j in range(sample_size):
        
        # Draw random vector
        vk = np.random.choice([-1, 1], size=n)

        # Update tk
        tk = tk + ((A @ vk) * vk)

        # Update qk
        qk = qk + (vk*vk)

        # Update diag_estimate
        diag_estimate = tk / qk

    return diag_estimate

In [ ]:
def naive_diag(A, sample_size=1000):
    """Naive unbiased estimator for the diagonal of a matrix, see [5].
    """

    # Get shape
    n = A.shape[0]

    diag_estimate = np.zeros(n)
    tk = np.zeros(n)
    qk = np.zeros(n)

    for j in range(sample_size):
        
        # Draw random vector
        vk = np.random.choice([-1, 1], size=n)

        # Update tk
        tk = tk + ((A @ vk) * vk)

        # Update qk
        qk = qk + (vk*vk)

        # Update diag_estimate
        diag_estimate = tk / qk

    return diag_estimate